In [18]:
# ==============================
# MULTI-DOMAIN AGENTIC AI SYSTEM (AUTO-DETECT DOMAIN)
# ==============================
STOPWORDS = {
    "is",
    "are",
    "was",
    "were",
    "the",
    "a",
    "an",
    "in",
    "on",
    "to",
    "of",
    "and",
    "with",
    "as",
    "for",
    "i",
    "my",
    "you",
    "he",
    "she",
    "it",
    "we",
    "they",
    "that",
    "this",
    "but",
    "can",
    "do",
    "does",
    "did",
    "at",
    "by",
    "if",
    "or",
    "be",
    "from",
    "about",
    "have",
    "has",
    "had",
    "am",
    "will",
    "shall",
    "would",
    "should",
    "could",
}

# --- KNOWLEDGE BASES ---


class LegalKnowledgeBase:
    def get_documents(self):
        return [
            {
                "title": "Employment Law",
                "content": "Firing someone due to pregnancy is illegal under the Pregnancy Discrimination Act.",
            },
            {
                "title": "Contract Breach",
                "content": "A breach of contract occurs when one party fails to fulfill its obligations.",
            },
            {
                "title": "Workplace Harassment",
                "content": "Employees have the right to report and be protected from harassment.",
            },
        ]


class FinanceKnowledgeBase:
    def get_documents(self):
        return [
            {
                "title": "Credit Score",
                "content": "A credit score represents a person's creditworthiness. Scores above 700 are considered good.",
            },
            {
                "title": "Investment Risk",
                "content": "Diversifying your portfolio reduces risk. High returns often come with high risk.",
            },
            {
                "title": "Loan Eligibility",
                "content": "Lenders consider income, credit score, and debt when approving loans.",
            },
        ]


class HealthKnowledgeBase:
    def get_documents(self):
        return [
            {
                "title": "Common Cold",
                "content": "Symptoms include sneezing, sore throat, runny nose, and mild cough.",
            },
            {
                "title": "Influenza",
                "content": "Symptoms include fever, chills, muscle aches, fatigue, and dry cough.",
            },
            {
                "title": "Migraine",
                "content": "A migraine involves a throbbing headache, nausea, and sensitivity to light.",
            },
        ]


# --- RETRIEVER (RAG) ---


class Retriever:

    def clean_text(self, text):
        return "".join(ch for ch in text if ch.isalnum() or ch.isspace())

    def retrieve(self, query, documents):
        clean_query = self.clean_text(query).lower()
        keywords = [word for word in clean_query.split() if word not in STOPWORDS]
        relevant = []

        for doc in documents:
            content = doc["content"].lower()
            score = sum(1 for word in keywords if word in content)
            if score >= 1:
                relevant.append(doc)

        return relevant


# --- MOCK LLM ---


class MockLLM:
    def generate(self, query, documents):
        if not documents:
            return f"""
QUERY: {query}

No relevant documents found.

Please consult a domain expert for accurate information.
"""
        summary = "\n".join(f"- {doc['title']}: {doc['content']}" for doc in documents)

        return f"""
QUERY: {query}

MATCHED INFORMATION:
{summary}

RECOMMENDATION:
This response is based on knowledge base entries related to your query.
Please verify this with a certified expert in the domain.
"""


# --- AGENT ---


class Agent:
    def __init__(self, knowledge_base, retriever, llm):
        self.kb = knowledge_base
        self.retriever = retriever
        self.llm = llm

    def run(self, query):
        print("🔍 Step 1: Retrieving documents...")
        docs = self.retriever.retrieve(query, self.kb.get_documents())
        print(f"✅ Found {len(docs)} relevant document(s).\n")

        print("🧠 Step 2: Generating response...")
        response = self.llm.generate(query, docs)

        print("\n📄 Step 3: Final Report:\n")
        print(response)


# --- DOMAIN CLASSIFIER ---


class DomainClassifier:
    def __init__(self):
        self.domain_keywords = {
            "legal": [
                "law",
                "contract",
                "fired",
                "harassment",
                "discrimination",
                "sue",
                "legal",
                "rights",
            ],
            "finance": [
                "loan",
                "credit",
                "investment",
                "bank",
                "score",
                "money",
                "interest",
                "finance",
            ],
            "health": [
                "fever",
                "headache",
                "cough",
                "pain",
                "cold",
                "symptom",
                "sick",
                "ill",
                "nausea",
                "health",
            ],
        }

    def clean_text(self, text):
        return "".join(ch for ch in text if ch.isalnum() or ch.isspace())

    def classify(self, query):
        clean_query = self.clean_text(query).lower()
        query_words = [word for word in clean_query.split() if word not in STOPWORDS]

        scores = {"legal": 0, "finance": 0, "health": 0}

        for domain, keywords in self.domain_keywords.items():
            scores[domain] = sum(1 for word in query_words if word in keywords)

        # Pick the domain with the highest score
        best_domain = max(scores, key=scores.get)
        return best_domain if scores[best_domain] > 0 else None


# --- MAIN ---


def main():
    print("🔧 Multi-Domain Agentic AI (Auto Domain Detection)")
    print("--------------------------------------------------")
    query = input("Enter your question: ").strip()
    print(f"User Input: {query}")

    # Classify domain
    classifier = DomainClassifier()
    domain = classifier.classify(query)

    if not domain:
        print("❌ Could not determine the domain of your question.")
        return

    print(f"🧭 Detected domain: {domain.upper()}")

    # Select correct KB
    if domain == "legal":
        kb = LegalKnowledgeBase()
    elif domain == "finance":
        kb = FinanceKnowledgeBase()
    elif domain == "health":
        kb = HealthKnowledgeBase()

    agent = Agent(kb, Retriever(), MockLLM())
    agent.run(query)


# --- RUN ---
main()

🔧 Multi-Domain Agentic AI (Auto Domain Detection)
--------------------------------------------------
User Input: Domain legal: fired for hiding pregnancy
🧭 Detected domain: LEGAL
🔍 Step 1: Retrieving documents...
✅ Found 1 relevant document(s).

🧠 Step 2: Generating response...

📄 Step 3: Final Report:


QUERY: Domain legal: fired for hiding pregnancy

MATCHED INFORMATION:
- Employment Law: Firing someone due to pregnancy is illegal under the Pregnancy Discrimination Act.

RECOMMENDATION:
This response is based on knowledge base entries related to your query.
Please verify this with a certified expert in the domain.

